In [2]:
import os
import pyspark
conf = pyspark.SparkConf()
# conf.set('spark.ui.proxyBase'
# , '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/04 21:58:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/04 21:58:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


/Users/poojaakki/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
# Read the csv files from datasets

#Fake news
df_fake = spark.read.option("header", "true").csv("/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/News _dataset/Fake.csv")
df_fake.show()

+--------------------+--------------------+-------+-----------------+
|               title|                text|subject|             date|
+--------------------+--------------------+-------+-----------------+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017|
| Racist Alabama C...|The number of cas...|   News|December 25, 2017|
| Fresh Off The Go...|Donald Trump spen...|   News|December 23, 2017|
| Trump Said Some ...|In the wake of ye...|   News|December 23, 2017|
| Former CIA Direc...|Many people have ...|   News|December 22, 2017|
| WATCH: Brand-New...|Just when you mig...|   News|December 21, 2017|
| Papa John’s Foun...|A centerpiece of ...|   News|December 21, 2017|
| WATCH: Paul Ryan..

In [4]:
#Truth news
df_true = spark.read.option("header", "true").csv("/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/News _dataset/True.csv")
#df_true.show()

In [5]:
# count of data and the. umber of coloumns
# print((df_fake.count(), len(df_fake.columns)))
# print((df_true.count(), len(df_true.columns)))

In [6]:
#Add the column for fake news and also a column for true news
from pyspark.sql.functions import lit

df_fake = df_fake.withColumn("Flag", lit("Fake_News"))
#df_fake.show()

In [7]:
df_true = df_true.withColumn("Flag", lit("True_News"))
# df_true.show()

In [8]:
#concat the two tables
df_concat = df_fake.unionByName(df_true, allowMissingColumns=True)
# df_concat.show()

In [9]:
#specifying the row number to each row

from pyspark.sql.functions import monotonically_increasing_id 

df_concat = df_concat.select("*").withColumn("id", monotonically_increasing_id())

# df_concat.show()

In [10]:
# print((df_concat.count(), len(df_concat.columns)))

In [11]:
#### has to randomly shuffle the data
import pyspark.sql.functions as F

df_concat = df_concat.select("*").orderBy(F.rand())
# df_concat.show()

In [12]:
# Remove the unwanted coloumn

df_concat =  df_concat.drop(df_concat["date"])
# df_concat

In [13]:
# Remove the title for analysis

df_concat =  df_concat.drop(df_concat["title"])
# df_concat

In [14]:
# In order to avaoid the confusion convert everything to lowercase
from pyspark.sql.functions import col, split, lower, regexp_replace, length

df_concat = df_concat\
.withColumn("text", regexp_replace(lower(col("text")), r"[^0-9a-z]", " "))\
.where(length("text")>0)

# df_concat

In [15]:
# remove stop words from it
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/poojaakki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
df_concat = df_concat.withColumn('words',F.split(F.col('text'),' '))
# df_concat

In [17]:
from pyspark.ml.feature import StopWordsRemover 

stopwordsRemovalFeature = StopWordsRemover(inputCol="words", 
                   outputCol="words without stop").setStopWords(stop)

In [18]:
from pyspark.ml import Pipeline

stopWordRemovalPipeline = Pipeline(stages=[stopwordsRemovalFeature])
pipelineFitRemoveStopWords = stopWordRemovalPipeline.fit(df_concat)

In [19]:
df_concat = pipelineFitRemoveStopWords.transform(df_concat)

In [20]:
# df_concat

In [21]:
# convert array of strings to strings
from pyspark.sql.functions import concat_ws

df_concat = df_concat.withColumn("text1", concat_ws(" ", df_concat["words without stop"]))
# df_concat

In [22]:
df_concat = df_concat.drop("words","words without stop")

df_concat

text,subject,Flag,id,text1
washington reute...,politicsNews,True_News,94489283131,washington reute...
berlin reuters ...,worldnews,True_News,111669151903,berlin reuters ...
reuters three...,politicsNews,True_News,85899347725,reuters three...
havana reuters ...,politicsNews,True_News,68719478171,havana reuters ...
white house couns...,News,Fake_News,1520,white house couns...
washington reute...,politicsNews,True_News,103079215130,washington reute...
one of the tenets...,News,Fake_News,17179869317,one tenets christ...
white house press...,News,Fake_News,2410,white house press...
washington reute...,politicsNews,True_News,68719477551,washington reute...
remote controlled...,Government News,Fake_News,34359741870,remote controlled...


In [23]:
df_concat = df_concat.drop("text")

df_concat

subject,Flag,id,text1
politicsNews,True_News,94489283131,washington reute...
worldnews,True_News,111669151903,berlin reuters ...
politicsNews,True_News,85899347725,reuters three...
politicsNews,True_News,68719478171,havana reuters ...
News,Fake_News,1520,white house couns...
politicsNews,True_News,103079215130,washington reute...
News,Fake_News,17179869317,one tenets christ...
News,Fake_News,2410,white house press...
politicsNews,True_News,68719477551,washington reute...
Government News,Fake_News,34359741870,remote controlled...


In [24]:
df_concat = df_concat.withColumnRenamed("text1","text")

df_concat

subject,Flag,id,text
politicsNews,True_News,94489283131,washington reute...
worldnews,True_News,111669151903,berlin reuters ...
politicsNews,True_News,85899347725,reuters three...
politicsNews,True_News,68719478171,havana reuters ...
News,Fake_News,1520,white house couns...
politicsNews,True_News,103079215130,washington reute...
News,Fake_News,17179869317,one tenets christ...
News,Fake_News,2410,white house press...
politicsNews,True_News,68719477551,washington reute...
Government News,Fake_News,34359741870,remote controlled...


In [25]:
# most frequent words in fake news

# import pyspark.sql.functions as F

# fake_words_freq_df = df_concat.withColumn("text", F.split("text", " ")) \
#     .groupBy("Flag") \
#     .agg(F.collect_list(F.col("text")).alias("text")) \
#     .withColumn("w", F.array_distinct(F.flatten("text"))) \
#     .withColumn("text", F.array_join(F.expr("filter(w, x -> size(filter(text, y -> array_contains(y, x))) = size(text))"), " ")) \
#     .drop("w")

# fake_words_freq_df

In [26]:
from nltk import tokenize

token_space = tokenize.WhitespaceTokenizer()

def counter(text, column_text, quantity):
    all_words = ' '.join([text for text in text[column_text]])
    token_phrase = token_space.tokenize(all_words)
    frequency = nltk.FreqDist(token_phrase)
    df_frequency = pd.DataFrame({"Word": list(frequency.keys()),
                                   "Frequency": list(frequency.values())})
    df_frequency = df_frequency.nlargest(columns = "Frequency", n = quantity)
    plt.figure(figsize=(12,8))
    ax = sns.barplot(data = df_frequency, x = "Word", y = "Frequency", color = 'blue')
    ax.set(ylabel = "Count")
    plt.xticks(rotation='vertical')
    plt.show()

In [27]:
# most frequent words in fake news

df_fake = df_concat.filter(df_concat["Flag"] == "Fake_News")

df_fake_text = df_fake.select("text")

# df_fake_text.show()

In [28]:
from pyspark.sql.functions import explode

df_fake_text = df_fake_text\
.withColumn("words", split(col("text"), " "))\
.where(length("text")>0)

df_freq_fake = df_fake_text\
.select(explode(col("words")).alias("word"))\
.groupBy("word").count()\

# df_freq_fake.orderBy(col("count").desc()).show()

In [29]:
# most frequent words in true news

df_true = df_concat.filter(df_concat["Flag"] == "True_News")

df_true_text = df_true.select("text")

# df_true_text.show()

In [30]:
df_true_text = df_true_text\
.withColumn("line", regexp_replace(lower(col("text")), r"[^0-9a-z]", " "))\
.withColumn("words", split(col("line"), " "))\
.where(length("line")>0)

# df_true_text

In [31]:
from pyspark.sql.functions import explode

df_fake_text = df_fake_text\
.withColumn("words", split(col("text"), " "))\
.where(length("text")>0)

df_freq_fake = df_fake_text\
.select(explode(col("words")).alias("word"))\
.groupBy("word").count()\

# df_freq_fake.orderBy(col("count").desc()).show()

#### Logistic Regression

In [32]:
df = df_concat

# df

In [33]:
from pyspark.sql.functions import when

df3 = df.withColumn("Flag", when(df.Flag == "True_News",1) \
      .when(df.Flag == "Fake_News",0) \
      .otherwise(df.Flag))

# df3

In [34]:
df3 = df3.withColumn("Flag",col("Flag").cast("int"))
df3.printSchema()
df3.count()

root
 |-- subject: string (nullable = true)
 |-- Flag: integer (nullable = true)
 |-- id: long (nullable = false)
 |-- text: string (nullable = false)



44898

In [35]:
# x values and Y values

x = df3.select('text').rdd.flatMap(lambda x: x).collect()
y = df3.select('Flag').rdd.flatMap(lambda x: x).collect()

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=.25)

In [37]:
x_train[0]

'american translation   baby nothin  wrong  good boy loved everyone   never mind brother wanted man world   pretty much everyone strapped suicide vest least stressful day right  seem like nice family ibrahim abdeslam  31  detonated suicide vest outside cafe comptoir voltaire  close scenes deadly bataclan concert theatre hall atrocities  friday night kill anyone brother mohamed  council worker  arrested brussels saturday third sibling  salah  one world wanted men becoming subject international man hunt abdeslam salah currently wanted man worldbut today abdeslam family defended sick actions said  surprised  blew  even though spent time war torn syria mother suggested suicide jacket may gone accident said could carried attack  stressed  speaking outside family home molenebeek  told reporters sure planned kill anyone said   plan  sure  fact suicide belt exploded without killing anyone says lot  even saw two days attacks  signs plans anything violent family admit ibrahim spent  long time  s

In [38]:
y_train[0]

0

In [39]:
#vectorize the text

from sklearn.feature_extraction.text import TfidfVectorizer

In [40]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [41]:
xv_train

<33673x104877 sparse matrix of type '<class 'numpy.float64'>'
	with 5477389 stored elements in Compressed Sparse Row format>

In [42]:
xv_train.toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [43]:
from sklearn.linear_model import LogisticRegression

In [44]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [45]:
LR.score(xv_test, y_test)

0.9879732739420936

In [55]:
pred_LR = LR.predict(xv_test)
pred_LR

array([0, 0, 1, ..., 0, 0, 0])

In [47]:
print(classification_report(y_test, pred_LR))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5810
           1       0.99      0.99      0.99      5415

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [48]:
import re, string
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [51]:
import pandas as pd
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    print("*****")
    print(new_xv_test)
    print(type(pred_LR[0]))
    print(type(pred_LR[0]))
    # pred_DT = DT.predict(new_xv_test)
    # pred_GBC = GBC.predict(new_xv_test)
    # pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \n".format(output_lable(pred_LR[0])))

In [52]:
news = str(input())
manual_testing(news)

 The following statements were posted to the verified Twitter accounts of U.S. President Donald Trump, @realDonaldTrump and @POTUS.  The opinions expressed are his own. Reuters has not edited the statements or confirmed their accuracy.  @realDonaldTrump : - Vanity Fair, which looks like it is on its last legs, is bending over backwards in apologizing for the minor hit they took at Crooked H. Anna Wintour, who was all set to be Amb to Court of St James’s & a big fundraiser for CH, is beside herself in grief & begging for forgiveness! [1024 EST] -- Source link: (bit.ly/2jBh4LU) (bit.ly/2jpEXYR)


*****
  (0, 101949)	0.24159989211014318
  (0, 99051)	0.14461019564675218
  (0, 98713)	0.17563313359496768
  (0, 96071)	0.06829499578042293
  (0, 95362)	0.04120098320105787
  (0, 94218)	0.07121244319202534
  (0, 88770)	0.21077403635538
  (0, 88462)	0.127034988721595
  (0, 87622)	0.08928785749971403
  (0, 84265)	0.07924792240558552
  (0, 79484)	0.041679050939101715
  (0, 77208)	0.21313728545600577
  (0, 74039)	0.03974160789705344
  (0, 73524)	0.1300508614184592
  (0, 73388)	0.0985523304355116
  (0, 68401)	0.1285780017253633
  (0, 61588)	0.13800930668429917
  (0, 57451)	0.3163725623600713
  (0, 56806)	0.10621887336705747
  (0, 56165)	0.12336171852581311
  (0, 56005)	0.05330697592223333
  (0, 55314)	0.15501949080015626
  (0, 54677)	0.051865006558304144
  (0, 49449)	0.09636624553816153
  (0, 44362)	0.09477692409278825
  (0, 41142)	0.1689040626059586
  (0, 38261)	0.14793124762564905
  (0, 37259)	0.17940803083507834
  (0, 37025)	0.08033355805935756
  (0, 34963)	0.10721585310873813
  (0, 34532